In [ ]:
import os
import sys
import numpy as np
from py.path import local
import warnings

sys.path.append('../..')
from seismiqb import SeismicGeometry
from seismiqb.src.geometry import export

warnings.filterwarnings("ignore")

In [ ]:
# Constants and parameters:
FORMATS = ['hdf5', 'qhdf5', 'blosc', 'qblosc']
CUBE_PATH = os.path.join(SAVING_DIR, 'tmp/', CUBE_NAME)

# Preparation

## Create a fake cube

In [ ]:
rng = np.random.default_rng(SEED)
data_array = rng.normal(0, 1000, CUBE_SHAPE).astype(np.float32)

In [ ]:
with open(os.path.join(SAVING_DIR, f'tmp/test_array_{DATESTAMP}.npy'), 'wb') as outfile:
    np.save(outfile, data_array)

In [ ]:
%%time
export.make_segy_from_array(array=data_array, path_segy=CUBE_PATH, zip_segy=False, sample_rate=2., delay=50, pbar='t')

In [ ]:
%%time
geometry_sgy = SeismicGeometry(path=CUBE_PATH, process=True, collect_stats=True, spatial=True, pbar='t')

## Check data loading

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_CDP
)

In [ ]:
%%time
_ = SeismicGeometry(
    path=CUBE_PATH,
    headers=SeismicGeometry.HEADERS_POST_FULL,
    index_headers=SeismicGeometry.INDEX_POST
)

## Conversion

In [ ]:
for f in FORMATS:
    _ = geometry_sgy.convert(format=f, quantize=False, store_meta=False, pbar='t')